1. 数据预处理（预估耗时：1-2小时）
任务：
解析 train.txt 和 test_without_label.txt 文件，建立文本和图像的配对数据集。
对文本数据进行清洗（如去除特殊字符、分词）并转为向量表示（如TF-IDF、Word2Vec或BERT嵌入）。
对图像数据进行标准化和特征提取（如通过预训练模型 ResNet/VGG 提取特征）。
划分训练集、验证集（建议 8:2 比例）。
输出：
预处理后的训练集和验证集。
可视化部分数据的分布（情感类别比例）。

文本预处理：清洗、分词，使用BERT或Word2Vec提取特征。
图像预处理：通过ResNet或VGG提取图像特征。
数据划分：将训练集划分为训练和验证集（8:2）。
可视化：展示情感类别的分布。


In [17]:

import pandas as pd
from PIL import Image
import os

# 读取训练集标签
train_labels = pd.read_csv('E:/Learning_material/junior/AI/5_FinalLab/dataset/train.txt', sep=' ', header=None, names=['guid', 'label'])

# 读取测试集，标签为空
test_guids = pd.read_csv('E:/Learning_material/junior/AI/5_FinalLab/dataset/test_without_label.txt', header=None, names=['guid', 'label'])

# 定义加载图像的函数
def load_image(guid):
    image_path = os.path.join('E:/Learning_material/junior/AI/5_FinalLab/dataset/data', f"{guid}.jpg")
    return Image.open(image_path)

def load_text(guid):
    text_path = os.path.join('E:/Learning_material/junior/AI/5_FinalLab/dataset/data', f"{guid}.txt")
    with open(text_path, 'r') as file:
        return file.read()

# load_image(1)
# load_text(1)

'How I feel today #legday #jelly #aching #gym \n'

文本预处理
清洗文本数据，去除特殊字符、分词（如使用 nltk 或 spaCy 库），然后将文本转化为向量表示。你可以使用：
TF-IDF：适用于较简单的文本特征提取。
Word2Vec/BERT：适用于更加复杂的情感分析任务，能捕捉上下文信息。
例如，使用 BERT 嵌入

In [18]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def encode_text(text):
    # 对文本进行分词并转化为 BERT 输入
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # 获取 BERT 模型输出
    with torch.no_grad():
        outputs = model(**inputs)
    # 提取最后一层的隐藏状态并做池化处理
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

 图像预处理
使用预训练模型（如 ResNet 或 VGG）提取图像特征

In [19]:
import torchvision.models as models
from torchvision import transforms
from PIL import Image

# 预处理图像
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 加载预训练的ResNet50模型
model = models.resnet50(pretrained=True)
model.eval()

# 特征提取函数
def extract_image_features(image):
    image_tensor = preprocess(image).unsqueeze(0)  # 扩展批量维度
    with torch.no_grad():
        features = model(image_tensor)
    return features.squeeze().detach().numpy()

e:\Learning_material\junior\AI\5_FinalLab\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\Learning_material\junior\AI\5_FinalLab\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
from sklearn.model_selection import train_test_split

# 假设你已经加载了文本向量和图像特征
# X_text = [encode_text(text) for text in train_labels['guid']]  # 用你的数据填充
X_text = [encode_text(load_text(guid)) for guid in train_labels['guid']]  # 通过 guid 获取文本
# X_image = [extract_image_features(load_image(guid)) for guid in train_labels['guid']]
X_image = [extract_image_features(load_image(guid)) for guid in train_labels['guid']]
y = train_labels['label'].values

X_train, X_val, y_train, y_val = train_test_split(list(zip(X_text, X_image)), y, test_size=0.2, random_state=42)

FileNotFoundError: [Errno 2] No such file or directory: 'E:/Learning_material/junior/AI/5_FinalLab/dataset/data\\guid,tag.txt'

In [24]:
from transformers import BertTokenizer, BertModel
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import os

# BERT模型准备（用于文本处理）
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # BERT模型输入文本并获取输出
    with torch.no_grad():
        outputs = model(**inputs)
    # 提取最后一层的隐藏状态并做池化处理
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

# ResNet模型准备（用于图像处理）
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

resnet_model = models.resnet50(pretrained=True)
resnet_model.eval()

def extract_image_features(image):
    image_tensor = preprocess(image).unsqueeze(0)  # 添加批量维度
    with torch.no_grad():
        features = resnet_model(image_tensor)
    return features.squeeze().detach().numpy()

# 加载文本和图像文件的路径
def load_text(guid):
    text_path = os.path.join('E:/Learning_material/junior/AI/5_FinalLab/dataset/data', f"{guid}.txt")
    with open(text_path, 'r') as file:
        return file.read()

def load_image(guid):
    image_path = os.path.join('E:/Learning_material/junior/AI/5_FinalLab/dataset/data', f"{guid}.jpg")
    return Image.open(image_path)

# 假设train_labels已经正确加载，train_labels['guid']是文件名的基础部分
X_text = []
X_image = []

for guid in train_labels['guid']:
    # 先加载文本
    text = load_text(guid)
    if text:  # 确保文本数据存在
        X_text.append(encode_text(text))  # 通过encode_text处理文本
    # 然后加载图像
    image = load_image(guid)
    X_image.append(extract_image_features(image))  # 通过extract_image_features提取图像特征

y = train_labels['label'].values  # 获取标签列

# 划分数据集
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(list(zip(X_text, X_image)), y, test_size=0.2, random_state=42)


e:\Learning_material\junior\AI\5_FinalLab\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\Learning_material\junior\AI\5_FinalLab\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


UnicodeDecodeError: 'gbk' codec can't decode byte 0xac in position 75: illegal multibyte sequence

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
train_labels['label'].value_counts().plot(kind='bar', color=['green', 'blue', 'red'])
plt.title('Emotion Label Distribution')
plt.xlabel('Emotion')
plt.ylabel('Frequency')
plt.show()